In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
from urllib import request
import json
from skimage import io

In [ ]:
#creates directory if not present
def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)
	return path

# retrives data at instanceURI
def make_labels(labels):
	lab_num = 0
	feat_num = 0
	for label in labels:
		label = json.loads(label)
		lab_num += 1
		im_objects = label['objects']
		for feature_dict in im_objects:
			feat_num += 1
			feat_type = feature_dict["value"]
			label_url = feature_dict["instanceURI"]
			file_name = f"obj{feat_num}-{feat_type}.png"
			dir_path = os.path.join("/content/drive/MyDrive/BTP/Results- Part-1/JSON to CSV file/Train & Test Data/Test-dataset", 'masks')
			label_dir = create_dir(os.path.join(dir_path, f"label{lab_num}"))
			label_path = os.path.join(label_dir, file_name)
			request.urlretrieve(label_url, label_path)


def get_shape(label_path):
	H = 0
	W = 0
	for feat in os.listdir(label_path):
		feat_path = os.path.join(label_path, feat)
		feat_array = cv2.imread(feat_path, cv2.IMREAD_GRAYSCALE)
		H, W = feat_array.shape
		break
	return (H, W)

def make_masks(dataset_path):
	mask_num = 0
	for label in os.listdir(os.path.join(dataset_path, 'masks')):
		mask_num += 1
		dir_name = f'label{mask_num}'
		label_path = os.path.join(dataset_path, 'masks', dir_name)
		H, W = get_shape(label_path)
		label_array = np.zeros((960, 1280, 3))
		dH = 960-H
		dW = 1280-W
		for feat in os.listdir(label_path):
			name, ext = feat.split(".")
			f_num, f_type = name.split("-")		
			feat_array = cv2.imread(os.path.join(label_path, feat), cv2.IMREAD_GRAYSCALE)/255 
			if(dH<0 or dW<0):
				feat_array = feat_array[:960, :1280]
			else:
				feat_array = cv2.copyMakeBorder(feat_array, 0, dH, 0, dW, cv2.BORDER_REFLECT)
			mask_array = np.zeros((960, 1280, 3))
			if(f_type == 'ductile'):
				mask_array[:, :, 0] = 255*np.logical_or(mask_array[:, :, 0], feat_array).astype(int)
			if(f_type == 'brittle'):
				mask_array[:, :, 1] = 255*np.logical_or(mask_array[:, :, 1], feat_array).astype(int)
			if(f_type == 'background'):
				mask_array[:, :, 2] = 255*np.logical_or(mask_array[:, :, 2], feat_array).astype(int)
			label_array += mask_array
			
		ind_unk = np.where((label_array[:, :, 0]+label_array[:, :, 1]+label_array[:, :, 2])==0)
		ind_unk = list(ind_unk)
		ind_unk.append(np.array([0]))
		ind_unk = tuple(ind_unk)
		label_array[ind_unk] = 255
		
		ind_amb = np.where((label_array[:, :, 0]+label_array[:, :, 1]+label_array[:, :, 2])>255)
		ind_amb1 = list(ind_amb)
		ind_amb1.append(np.array([0]))
		ind_amb2 = list(ind_amb)
		ind_amb2.append(np.array([1]))
		ind_amb3 = list(ind_amb)
		ind_amb3.append(np.array([2]))
		label_array[tuple(ind_amb1)] = 0
		label_array[tuple(ind_amb2)] = 0
		label_array[tuple(ind_amb3)] = 255
		file_name = f'{dir_name}.png'
		if file_name not in os.listdir(os.path.join(dataset_path, 'masks')):
			val = cv2.imwrite(os.path.join(dataset_path, 'masks', file_name), label_array)
			if val == True:
				print("image successfully saved on drive")
			else:
				print("image not saved on drive")



#retrives images from instanceURI
def make_images(images):
	for im_num in range(len(images)):
		im_url = images[im_num]
		file_name = f"image{im_num+1}.png"
		file_path = os.path.join('/content/drive/MyDrive/BTP/Results- Part-1/JSON to CSV file/Train & Test Data/Test-dataset', 'images')
		image_path = os.path.join(file_path, file_name)
		request.urlretrieve(im_url, image_path)


def crop_images(dataset):
	for im_num in range(1, len(os.listdir(os.path.join(dataset, 'images')))):
		file_name = f'image{im_num}.png'
		file_path = os.path.join(dataset, 'images')
		im_arr = cv2.imread(os.path.join(file_path, file_name), 0)
		H, W = im_arr.shape
		dH = 960-H
		dW = 1280-W
		if(dH<0 or dW<0):
			im_arr = im_arr[:960, :1280]
		else:
			im_arr = cv2.copyMakeBorder(im_arr, 0, dH, 0, dW, cv2.BORDER_REFLECT)
		cv2.imwrite(os.path.join(file_path, file_name), im_arr)


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/BTP/Results- Part-1/JSON to CSV file/Train & Test Data/Fractographic_Test_Data.csv')
labels = data['Label']
images = data['Labeled Data']


In [ ]:
make_labels(labels)
make_images(images)

In [ ]:
dataset_path = '/content/drive/MyDrive/BTP/Results- Part-1/JSON to CSV file/Train & Test Data/Test-dataset'
make_masks(dataset_path)

image successfully saved on drive
image successfully saved on drive
image successfully saved on drive


In [ ]:
crop_images(dataset_path)

In [ ]:
for im_num in range(1, len(os.listdir(os.path.join(dataset_path, 'images')))):
		file_name = f'image{im_num}.png'
		file_path = os.path.join(dataset_path, 'images')
		im_arr = cv2.imread(os.path.join(file_path, file_name), 0)
		print(im_arr.shape)
		

(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
(960, 1280)
